# 04 - AI Orchestration with Azure AI Search
**(Semantic Kernel / C# version)**

In this lab, we will do a deeper dive into using Azure AI Search as a vector store, the different search methods it supports and how you can use it as part of the Retrieval Augmented Generation (RAG) pattern for working with large language models.

## Create an Azure AI Search Vector Store in Azure

First, we will create an Azure AI Search service in Azure. We'll use the Azure CLI to do this, so you'll need to cut and paste the following commands into a terminal window.

**NOTE:** Before running the commands, replace the **`<INITIALS>`** with your own initials or some random characters, as we need to provide a unique name for the Azure AI Search service.

In [ ]:
// Execute the following commands using the Azure CLI to create the Azure AI Search resource in Azure.

RESOURCE_GROUP="azure-ai-search-rg"
LOCATION="westeurope"
NAME="ai-vectorstore-<INITIALS>"
az group create --name $RESOURCE_GROUP --location $LOCATION
az search service create -g $RESOURCE_GROUP -n $NAME -l $LOCATION --sku Basic --partition-count 1 --replica-count 1

Next, we need to find and update the following values in the `.env` file with the Azure AI Search **name**, **endpoint** and **admin key** values, which you can get from the Azure portal. You also need to provide an **index name** value. The index will be created during this lab, so you can use any name you like.

```
AZURE_AI_SEARCH_SERVICE_NAME = "<YOUR AZURE AI SEARCH SERVICE NAME - e.g. ai-vectorstore-xyz>"
AZURE_AI_SEARCH_ENDPOINT = "<YOUR AZURE AI SEARCH ENDPOINT URL - e.g. https://ai-vectorstore-xyz.search.windows.net"
AZURE_AI_SEARCH_INDEX_NAME = "<YOUR AZURE AI SEARCH INDEX NAME - e.g. ai-search-index>"
AZURE_AI_SEARCH_API_KEY = "<YOUR AZURE AI SEARCH ADMIN API KEY - e.g. get this value from the Azure portal>"
```

## Load the required .NET packages

This lab uses C#, so we will need to load the required .NET packages.

In [ ]:
// Add the Packages
#r "nuget: dotenv.net, 3.1.2"
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.16"
#r "nuget: Azure.Identity, 1.11.2"
#r "nuget: Azure.Search.Documents, 11.5.0-beta.5"

## Load environment variable values
As with previous labs, we'll use the values from the `.env` file in the root of this repository.

In [ ]:
using System.IO;
using dotenv.net;

// Read values from .env file
var envVars = DotEnv.Fluent()
    .WithoutExceptions()
    .WithEnvFiles("../../../.env")
    .WithTrimValues()
    .WithDefaultEncoding()
    .WithOverwriteExistingVars()
    .WithoutProbeForEnv()
    .Read();

var azure_openai_api_key = envVars["AZURE_OPENAI_API_KEY"].Replace("\"", "");
var azure_openai_endpoint = envVars["AZURE_OPENAI_ENDPOINT"].Replace("\"", "");
var azure_openai_embedding_deployment_name = envVars["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"].Replace("\"", "");
var azure_ai_search_name = envVars["AZURE_AI_SEARCH_SERVICE_NAME"].Replace("\"", "");
var azure_ai_search_endpoint = envVars["AZURE_AI_SEARCH_ENDPOINT"].Replace("\"", "");
var azure_ai_search_api_key = envVars["AZURE_AI_SEARCH_API_KEY"].Replace("\"", "");
var azure_ai_search_index_name = envVars["AZURE_AI_SEARCH_INDEX_NAME"].Replace("\"", "");

Console.WriteLine("This lab exercise will use the following values:");
Console.WriteLine($"Azure OpenAI Endpoint: {azure_openai_endpoint}");
Console.WriteLine($"Azure AI Search: {azure_ai_search_name}");

First, we will load the data from the movies.csv file and then extract a subset to load into the Azure AI Search index. We do this to help avoid the Azure OpenAI embedding limits and long loading times when inserting data into the index.

In [ ]:
using System.IO;

string path = @"./movies.csv";
string[] allMovies;
string[] movieSubset;
//
// Rather than load all 500 movies into Azure AI search, we will use a
// smaller subset of movie data to make things quicker. The more movies you load,
// the more time it will take for embeddings to be generated.
//
int movieSubsetCount = 50;
try
{
    if (File.Exists(path))
    {
        allMovies = File.ReadAllLines(path);
        movieSubset = allMovies.Skip(1).Take(movieSubsetCount).ToArray();
    }
}
catch (Exception e)
{
    Console.WriteLine("The process failed: {0}", e.ToString());
}

// Write out the results.
Console.WriteLine($"Loaded {movieSubset.Length} movies.");

During this lab, we will need to work with embeddings. We use embeddings to create a vector representation of a piece of text. We will need to create embeddings for the documents we want to store in our Azure AI Search index and also for the queries we want to use to search the index. We will create an Azure OpenAI client to do this.

In [ ]:
using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;

OpenAIClient azureOpenAIClient = new OpenAIClient(new Uri(azure_openai_endpoint),new AzureKeyCredential(azure_openai_api_key));

## Create an Azure AI Search index and load movie data

Next, we'll step through the process of configuring an Azure AI Search index to store our movie data and then loading the data into the index. 

In [ ]:
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;
using Microsoft.Extensions.Configuration;

// These variables store the names of Azure AI Search profiles and configs that we will use

string vectorSearchHnswProfile = "movies-vector-profile";
string vectorSearchHnswConfig = "movies-vector-config";
string semanticSearchConfigName = "movies-semantic-config";

// The OpenAI embeddings model has 1,536 dimensions

int modelDimensions = 1536;

When configuring an Azure AI Search index, we need to specify the fields we want to store in the index and the data types for each field. These match the fields in the movie data, containing values such as the movie title, genre, year of release and so on.

**NOTE:** It is possible just to use Azure AI Search as a vector store only, in which case we probably wouldn't need to define all of the index fields below. However, in this lab, we're also going to demonstrate Hybrid Search, a feature which makes use of both traditional keyword based search in combination with vector search.

In [ ]:
IList<SearchField> fields = new List<SearchField>();
fields.Add(new SimpleField("id", SearchFieldDataType.String) { IsKey = true, IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("title", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("overview", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("genre", false) { IsFilterable = true, IsSortable = true, IsFacetable = true });
fields.Add(new SearchableField("tagline", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("release_date", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("popularity", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("vote_average", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("vote_count", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("runtime", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("revenue", false) { IsFilterable = true, IsSortable = true });
fields.Add(new SearchableField("original_language", false) { IsFilterable = true, IsSortable = true });

To use Azure AI Search as a vector store, we will need to define a field to hold the vector representaion of the movie data. We indicate to Azure AI Search that this field will contain vector data by providing details of the vector dimensions and a profile. We'll also define the vector search configuration and profile with default values.

In [ ]:
fields.Add(new SearchField("vector", SearchFieldDataType.Collection(SearchFieldDataType.Single))
{
    IsSearchable = true,
    VectorSearchDimensions = modelDimensions,
    VectorSearchProfile = vectorSearchHnswProfile,
});

VectorSearch vectorSearch = new VectorSearch();
vectorSearch.Algorithms.Add(new HnswVectorSearchAlgorithmConfiguration(vectorSearchHnswConfig));
vectorSearch.Profiles.Add(new VectorSearchProfile(vectorSearchHnswProfile, vectorSearchHnswConfig));

We're going to be using Semantic Ranking, a feature of Azure AI Search that improves search results by using language understanding to rerank the search results. We configure Semantic Settings to help the ranking model understand the movie data, by telling it which fields contain the movie title, which fields contain keywords and which fields contain general free text content.

In [ ]:
SemanticSettings semanticSettings = new SemanticSettings();
semanticSettings.Configurations.Add(new SemanticConfiguration(semanticSearchConfigName, new PrioritizedFields()
{
    TitleField = new SemanticField() { FieldName = "title" },
    KeywordFields = { new SemanticField() { FieldName = "genre" } },
    ContentFields = { new SemanticField() { FieldName = "title" },
                      new SemanticField() { FieldName = "overview" },
                      new SemanticField() { FieldName = "tagline" },
                      new SemanticField() { FieldName = "genre" },
                      new SemanticField() { FieldName = "release_date" },
                      new SemanticField() { FieldName = "popularity" },
                      new SemanticField() { FieldName = "vote_average" },
                      new SemanticField() { FieldName = "vote_count" },
                      new SemanticField() { FieldName = "runtime" },
                      new SemanticField() { FieldName = "revenue" },
                      new SemanticField() { FieldName = "original_language" }
    }
}));

Finally, we'll go ahead and create the index by creating an instance of the `SearchIndex` class and adding the keyword and vectors fields and the semantic search profile.

In [ ]:
// Setup SearchIndex
SearchIndex searchIndex = new SearchIndex(azure_ai_search_index_name);
searchIndex.VectorSearch = vectorSearch;
searchIndex.SemanticSettings = semanticSettings;
searchIndex.Fields = fields;

// Create an Azure AI Search index client and create the index.
AzureKeyCredential indexCredential = new AzureKeyCredential(azure_ai_search_api_key);
SearchIndexClient indexClient = new SearchIndexClient(new Uri(azure_ai_search_endpoint), indexCredential);
indexClient.CreateOrUpdateIndex(searchIndex);

Console.WriteLine($"Index {azure_ai_search_index_name} created.");

The index is now ready, so next we need to prepare the movie data to load into the index.

**NOTE**: During this phase, we send the data for each movie to an Azure OpenAI embeddings model to create the vector data. This may take some time due to rate limiting in the API.

In [ ]:
using System.Text.RegularExpressions;

// Loop through all of the movies and create an array of documents that we can load into the index.
List<SearchDocument> movieDocuments = new List<SearchDocument>();
for (int i = 0; i < movieSubset.Length; i++) 
{
    // Extract the data for the movie from the CSV file into variables
    var values = Regex.Split(movieSubset[i], ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)");
    var movieId = values[0];
    var movieTitle = values[2];
    var movieOverview = Regex.Replace(values[8], "^\"|\"$", "");
    var movieGenre = values[7].Substring(2, values[7].Length - 4);
    var movieTagline = Regex.Replace(values[11], "^\"|\"$", "");
    var movieReleaseDate = values[4];
    var movieRuntime = values[10];
    var moviePopularity = values[3];
    var movieVoteAverage = values[5];
    var movieVoteCount = values[6];
    var movieRevenue = values[9];
    var movieLanguage = values[1];
    
    // Take the entire set of data for the movie and generate the embeddings
    IEnumerable<string> content = new List<string>() { movieSubset[i] };
    Response<Embeddings> contentEmbeddings = azureOpenAIClient.GetEmbeddings(new EmbeddingsOptions(azure_openai_embedding_deployment_name, content));

    // Add the movie data and embeddings to a search document
    SearchDocument document = new SearchDocument();
    document.Add("id", movieId.Substring(0, movieId.Length - 2));
    document.Add("title", movieTitle);
    document.Add("overview", movieOverview);
    document.Add("genre", movieGenre);
    document.Add("tagline", movieTagline);
    document.Add("release_date", movieReleaseDate);
    document.Add("runtime", movieRuntime);
    document.Add("popularity", moviePopularity);
    document.Add("vote_average", movieVoteAverage);
    document.Add("vote_count", movieVoteCount);
    document.Add("revenue", movieRevenue);
    document.Add("original_language", movieLanguage);
    document.Add("vector", contentEmbeddings.Value.Data[0].Embedding.ToArray());
    movieDocuments.Add(new SearchDocument(document));
    Console.WriteLine($"Movie {movieTitle} added.");
}

Console.WriteLine($"New SearchDocument structure with embeddings created for {movieDocuments.Count} movies.");

We can write out the contents of one of the documents to see what it looks like. You can see that it contains the movie data at the top and then a long array containing the vector data.

In [ ]:
Console.WriteLine(movieDocuments[0]);

Now we have the movie data stored in the correct format, so let's load it into the Azure AI Search index we created earlier.

In [ ]:
SearchClient searchIndexClient = indexClient.GetSearchClient(azure_ai_search_index_name);
IndexDocumentsOptions options = new IndexDocumentsOptions { ThrowOnAnyError = true };
searchIndexClient.IndexDocuments(IndexDocumentsBatch.Upload(movieDocuments), options);

Console.WriteLine($"Successfully loaded {movieDocuments.Count} movies into Azure AI Search index.");

## Vector store searching using Azure AI Search

We've loaded the movies into Azure AI Search, so now let's experiment with some of the different types of searches you can perform.

First we'll just perform a simple keyword search.

In [ ]:
var query = "hero";

SearchOptions searchOptions = new SearchOptions
{
    Size = 5,
    Select = { "title", "genre" },
    SearchMode = SearchMode.Any,
    QueryType = SearchQueryType.Simple,
};

SearchResults<SearchDocument> response = searchIndexClient.Search<SearchDocument>(query, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Movie: {result.Document["title"]}");
    Console.WriteLine($"Genre: {result.Document["genre"]}");
    Console.WriteLine("----------");
};

We get some results, but they're not necessarily movies about heroes. It could be that there is some text in the index for these results that relates to the word "hero". For example, the description might mention "heroic deeds" or something similar.

Let's now try the same again, but this time we'll ask a question instead of just searching for a keyword.

In [ ]:
var query = "What are the best movies about superheroes?";

SearchResults<SearchDocument> response = searchIndexClient.Search<SearchDocument>(query, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Movie: {result.Document["title"]}");
    Console.WriteLine($"Genre: {result.Document["genre"]}");
    Console.WriteLine("----------");
};

As before, you will likely get mixed results. Some of the movies returned could be about heroes, but others may not be. This is because the search is still based on keywords.

Next, let's try a vector search.

In [ ]:
var query = "What are the best movies about superheroes?";

// Convert the query to an embedding
float[] queryEmbedding = azureOpenAIClient.GetEmbeddings(new EmbeddingsOptions(azure_openai_embedding_deployment_name, new List<string>() { query })).Value.Data[0].Embedding.ToArray();

// This time we will set the search type to Semantic and we'll pass in the embedded version of the query text and parameters to configure the vector search.
SearchOptions searchOptions = new SearchOptions
{
    QueryType = SearchQueryType.Semantic,
    SemanticConfigurationName = semanticSearchConfigName,
    VectorQueries = { new RawVectorQuery() { Vector = queryEmbedding, KNearestNeighborsCount = 5, Fields = { "vector" } } },
    Size = 5,
    Select = { "title", "genre" },
};

// Note the `null` value for the query parameter. This is because we're not sending the query text to Azure AI Search. We're sending the embedded version of the query text instead.
SearchResults<SearchDocument> response = searchIndexClient.Search<SearchDocument>(null, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Movie: {result.Document["title"]}");
    Console.WriteLine($"Genre: {result.Document["genre"]}");
    Console.WriteLine($"Score: {result.Score}");
    //Console.WriteLine($"Reranked Score: {result.RerankerScore}");
    Console.WriteLine("----------");
};

It's likely that the raw vector search didn't return exactly what you were expecting. You were probably expecting a list of superhero movies, but now we're getting a list of movies that are **similar** to the vector we provided. Some of these may be hero movies, but others may not be. The vector search is returning the nearest neighbours to the vector we provided, so it's possible that at least one of the results is a superhero movie, and the others are similar to that movie in some way.

So, both the keyword search and the vector search have their limitations. The keyword search is limited to the keywords in the index, so it's possible that we might miss some movies that are about heroes. The vector search is limited to returning the nearest neighbours to the vector we provide, so it's possible that we might get some movies that are not about heroes.

## Hybrid search using Azure AI Search

To overcome the limitations of both keyword search and vector search, we can use a combination of both. This is known as Hybrid Search. Let's run the same query again, but this time we'll use Hybrid Search.

The only significant difference is that this time we will submit both the original query text and the embedding vector to Azure AI Search. Azure AI Search will then use both the query text and the vector to perform the search and combine the results.

In [ ]:
SearchResults<SearchDocument> response = searchIndexClient.Search<SearchDocument>(query, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Movie: {result.Document["title"]}");
    Console.WriteLine($"Genre: {result.Document["genre"]}");
    Console.WriteLine($"Score: {result.Score}");
    Console.WriteLine($"Reranked Score: {result.RerankerScore}");
    Console.WriteLine("----------");
};

Hopefully, you'll now see a much better set of results. Performing a hybrid search has allowed us to combine the benefits of both keyword search and vector search. But also, Azure AI Search performs a further step when using hybrid search. It makes use of a Semantic Ranker to further improve the search results. The Semantic Ranker uses a language understanding model to understand the query text and the documents in the index and then uses this information to rerank the search results. So, after performing the keyword and vector search, Azure AI Search will then use the Semantic Ranker to re-order the search results based on the context of the original query.

In the results above, you can see a `Reranked Score`. This is the score that has been calculated by the Semantic Ranker. The `Score` is the score calculated by the keyword and vector search. You'll note that the results are returned in the order determined by the reranked score.

## Bringing it All Together with Retrieval Augmented Generation (RAG) + Semantic Kernel (SK)

Now that we have our Vector Store setup and data loaded, we are now ready to implement the RAG pattern using AI Orchestration. At a high-level, the following steps are required:
1. Ask the question
2. Create Prompt Template with inputs
3. Get Embedding representation of inputted question
4. Use embedded version of the question to search Azure AI Search (ie. The Vector Store)
5. Inject the results of the search into the Prompt Template & Execute the Prompt to get the completion

**NOTE:** Semantic Kernel has a Semantic Memory feature, as well as a separate Kernel Memory project, that both allow you to store and retrieve information from a vector store. These are both currently in preview / experimental states. In the following code, we're implementing the feature using our own code to firstly retrieve results from Azure AI Search, then add those search results to the query we send to Azure OpenAI.

First, let's setup the packages and load the values from the `.env` file.

In [ ]:
// Add the Packages
#r "nuget: dotenv.net, 3.1.2"
#r "nuget: Microsoft.SemanticKernel, 1.10.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.10.0"
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.16"
#r "nuget: Azure.Identity, 1.11.2"
#r "nuget: Azure.Search.Documents, 11.5.0-beta.5"
#r "nuget: Microsoft.Extensions.Logging, 7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 7.0.0"

using System.IO;
using dotenv.net;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;
using Microsoft.Extensions.Configuration;

// Read values from .env file
var envVars = DotEnv.Fluent()
    .WithoutExceptions()
    .WithEnvFiles("../../../.env")
    .WithTrimValues()
    .WithDefaultEncoding()
    .WithOverwriteExistingVars()
    .WithoutProbeForEnv()
    .Read();

var azure_openai_api_key = envVars["AZURE_OPENAI_API_KEY"].Replace("\"", "");
var azure_openai_endpoint = envVars["AZURE_OPENAI_ENDPOINT"].Replace("\"", "");
var azure_openai_completion_deployment_name = envVars["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"].Replace("\"", "");
var azure_openai_embedding_deployment_name = envVars["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"].Replace("\"", "");
var azure_ai_search_name = envVars["AZURE_AI_SEARCH_SERVICE_NAME"].Replace("\"", "");
var azure_ai_search_endpoint = envVars["AZURE_AI_SEARCH_ENDPOINT"].Replace("\"", "");
var azure_ai_search_api_key = envVars["AZURE_AI_SEARCH_API_KEY"].Replace("\"", "");
var azure_ai_search_index_name = envVars["AZURE_AI_SEARCH_INDEX_NAME"].Replace("\"", "");

Console.WriteLine("This lab exercise will use the following values:");
Console.WriteLine($"Azure OpenAI Endpoint: {azure_openai_endpoint}");
Console.WriteLine($"Azure AI Search: {azure_ai_search_name}");

We'll setup Semantic Kernel with an Azure OpenAI service.

In [ ]:
var builder = Kernel.CreateBuilder();
builder.Services.AddAzureOpenAIChatCompletion(azure_openai_completion_deployment_name, azure_openai_endpoint, azure_openai_api_key);
var kernel = builder.Build();

Now we need to define a prompt template. In this template, we'll give the model the `System` instructions telling it to answer the users question using only the data provided. We'll then inject the search results that we'll later get from Azure AI Search into the `searchResults` variable. The `User` instructions will be the question that the user asks.

In [ ]:
var askQuery = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Name = "askQuery",
        Description = "Ask Azure OpenAI a question using results from Azure AI Search.",
        Template = @"System: Answer the user's question using only the movie data below. Do not use any other data. Provide detailed information about the synopsis of the movie. {{$searchResults}}
        User: {{$originalQuestion}}
        Assistant: ",
        TemplateFormat = "semantic-kernel",
        InputVariables = [
            new() { Name = "originalQuestion", Description = "The query we sent to Azure AI Search.", IsRequired = true },
            new() { Name = "searchResults", Description = "The results retrieved from Azure AI Search.", IsRequired = true }
        ],
        ExecutionSettings = {
            { "default", new OpenAIPromptExecutionSettings() {
                MaxTokens = 1000,
                Temperature = 0.1,
                TopP = 0.5,
                PresencePenalty = 0.0,
                FrequencyPenalty = 0.0
            } }
        }
    }
);

Next we define our question and send the question to an embedding model to generate the vector representation of the question.

In [ ]:
var query = "What are the best movies about superheroes?";

OpenAIClient azureOpenAIClient = new OpenAIClient(new Uri(azure_openai_endpoint),new AzureKeyCredential(azure_openai_api_key));
float[] queryEmbedding = azureOpenAIClient.GetEmbeddings(new EmbeddingsOptions(azure_openai_embedding_deployment_name, new List<string>() { query })).Value.Data[0].Embedding.ToArray();

Let's prepare the `SearchOptions` to carry out a vector search.

In [ ]:
string semanticSearchConfigName = "movies-semantic-config";

SearchOptions searchOptions = new SearchOptions
{
    QueryType = SearchQueryType.Semantic,
    SemanticConfigurationName = semanticSearchConfigName,
    VectorQueries = { new RawVectorQuery() { Vector = queryEmbedding, KNearestNeighborsCount = 5, Fields = { "vector" } } },
    Size = 5,
    Select = { "title", "genre" },
};

Now we can go ahead and perform the search. As noted previously, we'll provide both the original question text and the vector representation of the text in order to perform a hybrid search with semantic ranking.

In [ ]:
AzureKeyCredential indexCredential = new AzureKeyCredential(azure_ai_search_api_key);
SearchIndexClient indexClient = new SearchIndexClient(new Uri(azure_ai_search_endpoint), indexCredential);
SearchClient searchClient = indexClient.GetSearchClient(azure_ai_search_index_name);

//Perform the search
SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(query, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

The results of the search are now in the `results` variable. We'll iterate through them and turn them into a string that we can inject into the prompt template. When finished, we display these results so that you can see what is going to be injected into the template.

In [ ]:
StringBuilder stringBuilderResults = new StringBuilder();
foreach (SearchResult<SearchDocument> result in results)
{
    stringBuilderResults.AppendLine($"{result.Document["title"]}");
};

Console.WriteLine(stringBuilderResults.ToString());

Finally, we can inject the search results into the prompt template and execute the prompt to get the completion.

In [ ]:
var completion = await kernel.InvokeAsync(askQuery, new () { { "originalQuestion", query }, { "searchResults", stringBuilderResults.ToString() }});

Console.WriteLine(completion);

## Next Section

📣 [Deploy AI](../../04-deploy-ai/README.md)